In [1]:
import libtorrent as lt
import time
from IPython.display import display
import ipywidgets as widgets

In [2]:
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

In [3]:
params = {
    "save_path": "/home/frederick/Documents/GitHub/udemy_download/Torrent",
    "ti": lt.torrent_info("8B3D44703C08E0C2EF810AEFCD63151ADD145D51.torrent")
}
downloads.append(ses.add_torrent(params))

In [4]:
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…